In [ ]:
# !pip install --quiet openai
# !pip install --quiet python-dotenv

# Setting up configuration

In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv("../.env")

client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'), base_url='https://api.openai.com/v1')

In [5]:
def llm_get_response(prompt):
    """
    Function to get a response from the OpenAI API.
    """
    try:
        response = client.chat.completions.create(
            model = "gpt-4o",
            messages=[
                {"role": "system", "content": "You are a helpful assistant"},
                {"role": "user", "content": prompt},
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error: {e}")
        return None

In [6]:
prompt = "What is the capital of France"
response = llm_get_response(prompt)
if response:
    print(f"Response: {response}")

Response: The capital of France is Paris.


# Prompt Engineering Patterns & Templates
This notebook demonstrates various prompt engineering techniques using the theme:
> **Tell me more about generative AI**

We'll cover basic prompts, zero-shot, one-shot, few-shot, chain-of-thought, ReAct, and advanced techniques.

## 1. Basic Prompt
A straightforward instruction to elicit information.

In [8]:
basic_prompt = "Tell me more about generative AI."

response = llm_get_response(basic_prompt)
if response:
    print(f"Response: {response}")

Response: Generative AI refers to a class of artificial intelligence techniques specifically designed to create new content. This content can be in the form of text, images, music, code, or even more complex structures. The term "generative" is used because these models generate new data based on patterns they have learned from existing data.

Here are some key concepts and components of generative AI:

1. **Generative Models:** These are the backbone of generative AI. Common types include Generative Adversarial Networks (GANs), Variational Autoencoders (VAEs), and Transformer-based models like GPT (Generative Pre-trained Transformer).

   - **Generative Adversarial Networks (GANs):** GANs consist of two networks—a generator and a discriminator—competing against each other. The generator creates data, while the discriminator evaluates its authenticity. Through this adversarial process, GANs can produce highly realistic data, particularly images.

   - **Variational Autoencoders (VAEs):

## 2. Zero-Shot Prompt
Providing context or role without examples.

In [ ]:
zero_shot_prompt = (
    "You are an expert in AI. "  
    "Answer the following question concisely: 'Tell me more about generative AI.'"
)

response = llm_get_response(zero_shot_prompt)
if response:
    print(f"Response: {response}")

## 3. One-Shot Prompt
Including a single example to guide the model.

In [ ]:
one_shot_prompt = (
    "Example:\n"  
    "Q: What is language modeling?\n"
    "A: Language modeling is the task of predicting the next word in a sequence...\n\n"
    "Now answer this:\n"
    "Q: Tell me more about generative AI.\n"
    "A:"
)

response = llm_get_response(one_shot_prompt)
if response:
    print(f"Response: {response}")

## 4. Few-Shot Prompt
Providing multiple examples to improve consistency.

In [ ]:
few_shot_prompt = (
    "Example 1:\n"
    "Q: What is reinforcement learning?\n"
    "A: Reinforcement learning is...\n\n"
    "Example 2:\n"
    "Q: Define transfer learning.\n"
    "A: Transfer learning refers to...\n\n"
    "Now answer:\n"
    "Q: Tell me more about generative AI.\n"
    "A:"
)

response = llm_get_response(few_shot_prompt)
if response:
    print(f"Response: {response}")

## 5. Chain-of-Thought Prompt
Encourage the model to show its reasoning steps.

In [ ]:
cot_prompt = (
    "Explain step by step how generative AI models work. "
    "Then summarize the key points.\n"
    "Answer:"
)

response = llm_get_response(cot_prompt)
if response:
    print(f"Response: {response}")

## 6. ReAct Prompt
Combining reasoning and actions (e.g., API calls, tool use).

In [ ]:
react_prompt = (
    "You are an AI assistant. Use the following tools: [search(query), calculate(expression)].\n"
    "When asked 'Tell me more about generative AI', think step-by-step and if needed, call search.\n"
    "Thought: What is generative AI?\n"
    "Action: search('generative AI definition')\n"
    "Observation: ... results ...\n"
    "Final Answer:"
)

response = llm_get_response(react_prompt)
if response:
    print(f"Response: {response}")

## 7. Advanced Prompting Techniques
- **Self-Consistency**: sample multiple reasoning paths and pick the most consistent answer.
- **Tree of Thoughts**: explore multiple reasoning branches.
- **Dynamic Prompting**: adapt prompt based on intermediate responses.


### Self-Consistency Sampling
Rather than taking a single reasoning path, you sample multiple chain-of-thought generations and then aggregate or vote on the most consistent answer.


In [9]:
from collections import Counter

def self_consistency(prompt: str, samples: int = 5) -> str:
    """
    Ask the model to “think” step-by-step multiple times, then pick
    the most frequent final answer.
    """
    responses = []
    for i in range(samples):
        # we nudge the model to include reasoning + final answer
        full_prompt = (
            "Explain step by step, then give a concise answer.\n"
            f"Question: {prompt}\nAnswer:"
        )
        resp = llm_get_response(full_prompt)
        # assume the last line is the short answer
        answer = resp.strip().splitlines()[-1]
        responses.append(answer)
    # majority vote
    most_common = Counter(responses).most_common(1)[0][0]
    return most_common

if __name__ == "__main__":
    question = "Tell me more about generative AI."
    consensus = self_consistency(question, samples=7)
    print("Self-Consistency Answer:\n", consensus)


Self-Consistency Answer:
 Generative AI refers to machine learning models that can create new content, such as text, images, or music, by learning patterns from existing data. It's powered by models like GANs, VAEs, and transformers. Applications span from creative to functional uses, including text generation, art creation, and music composition, though they raise ethical and quality concerns. Generative AI continues to evolve, promising to reshape various industries.


### Tree of Thoughts
You structure a tree of intermediate “thoughts” (hypotheses), explore each branch, then select the best path.

In [10]:
def tree_of_thoughts(root_prompt: str, branches: list[list[str]]) -> str:
    """
    branches: a list of thought-lists, e.g.
      [
        ["Define generative AI", "Explain core architectures"],
        ["List use-cases", "Discuss ethical issues"]
      ]
    """
    best_answer = None
    best_score = -1.0  # you could use a model-provided score or heuristic
    for thought_path in branches:
        # build a multi-step prompt
        prefix = "\n".join(f"Thought: {t}" for t in thought_path)
        full_prompt = f"{prefix}\nQuestion: {root_prompt}\nAnswer:"
        resp = llm_get_response(full_prompt)

        # (Optional) ask model to score its own answer
        score_prompt = (
            f"Rate the following answer 0–1 for correctness and completeness:\n\n{resp}\n\nScore:"
        )
        score_resp = llm_get_response(score_prompt)
        try:
            score = float(score_resp.strip())
        except:
            score = 0.0

        if score > best_score:
            best_score = score
            best_answer = resp

    return best_answer

if __name__ == "__main__":
    question = "Tell me more about generative AI."
    paths = [
        ["Define generative AI", "Explain transformer architectures"],
        ["Describe common applications", "Highlight recent advances"]
    ]
    best = tree_of_thoughts(question, paths)
    print("Best Tree-of-Thoughts Answer:\n", best)


Best Tree-of-Thoughts Answer:
 Generative AI refers to a category of artificial intelligence systems that are designed to generate data or content that resembles human-created media. These systems can produce text, images, music, and other forms of media by learning patterns and structures from existing datasets and then using these patterns to create new, similar outputs.

One of the most prominent techniques used in generative AI is the transformer architecture. Transformer models, which include well-known instances such as OpenAI's GPT (Generative Pre-trained Transformer), are built on the concept of attention mechanisms. These mechanisms allow models to weigh the importance of different words (or other elements) when generating new content, thus enabling them to understand and create nuanced, contextually-relevant outputs.

Transformers have revolutionized generative AI by allowing for massively parallel processing and an ability to capture long-range dependencies in data, making t

### Dynamic Prompting
You iteratively adjust the prompt based on the model’s intermediate output (e.g., ask follow-up clarifications or refine instructions).

In [11]:
def dynamic_prompting(initial: str, feedback_steps: int = 2) -> str:
    """
    1. Send an initial prompt.
    2. Analyze the response; ask a targeted follow-up.
    3. Incorporate feedback and repeat.
    """
    prompt = initial
    for step in range(feedback_steps):
        resp = llm_get_response(prompt)
        print(f"Step {step} response:\n{resp}\n")

        # Generate a follow-up instruction
        followup = (
            "Based on the previous answer, please elaborate specifically on "
            "the key challenges of implementing generative AI in production."
        ) if step == 0 else (
            "Great—now summarize your points in a bulleted list."
        )

        # Build new prompt with context
        prompt = (
            f"{resp}\n\n"
            f"### Follow-up:\n{followup}\nAnswer:"
        )

    # final response
    final = llm_get_response(prompt)
    return final

if __name__ == "__main__":
    start = "Tell me more about generative AI."
    result = dynamic_prompting(start, feedback_steps=2)
    print("Dynamic Prompting Final Answer:\n", result)


Step 0 response:
Generative AI refers to a subset of artificial intelligence that focuses on creating new content or data, often resembling or inspired by existing datasets. Unlike traditional AI systems that focus on analyzing or classifying existing data, generative AI models are designed to generate new examples. Here are some key aspects and applications of generative AI:

1. **Underlying Technologies:**

   - **Generative Adversarial Networks (GANs):** Introduced by Ian Goodfellow in 2014, GANs consist of two neural networks—a generator and a discriminator—that are trained simultaneously. The generator creates samples, and the discriminator evaluates them against real data. The generator improves its output to fool the discriminator over time.

   - **Variational Autoencoders (VAEs):** VAEs are used for generating new data by learning a compressed latent representation of data which enables generation of new samples.

   - **Transformers:** Originally designed for tasks like langu